# Evaluation of Recommender Systems


Note: this lab has been tested with Python 3.10. We recommend using the same Python version if there are problems with libraries used in this lab.

In [106]:
# Load data generated in W6 Lab or the provided data splits (see Absalon, W7 Lab)
import pandas as pd
import random
import numpy as np
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from surprise import accuracy
from surprise import SVD

train_data = pd.read_pickle("train_dataframe.pkl")
test_data = pd.read_pickle("test_dataframe.pkl")
top_items = pd.read_csv('top_items.csv')

## Step 1
Specifically,
let us evaluate all your recommender models from Week 7 on the preprocessed
test data split studied in Week 6. You need to:
• Measure the error of the system’s predicted likelihood of rating for the
items (Root Mean Square Error, RMSE)4
.
• Discuss the limitations of this metric.



In [107]:
reader = Reader(rating_scale=(1, 5))
training_matrix = Dataset.load_from_df(train_data[['user_id', 'item_id', 'rating']], reader)
trainset = training_matrix.build_full_trainset()
unobserved_ratings = trainset.build_anti_testset()


### KNN

In [108]:
sim_options = {'name': 'pearson',
               'user_based': True
               }
algo_knn = KNNWithMeans(k= 6, 
                    random_state=0,
                    sim_options= sim_options,
                    verbose=False)

trainset = training_matrix.build_full_trainset()# includes the entire dataset for training
algo_knn.fit(trainset)

pred_KNN = algo_knn.test(unobserved_ratings)
rmse_score_knn = accuracy.rmse(pred_KNN)

print(f"For KNN model, the RMSE: {rmse_score_knn}")

RMSE: 0.5344
For KNN model, the RMSE: 0.5343510804831747


### SVD

In [109]:
algo_SVD = SVD(n_factors=30, n_epochs=20, random_state=0)
algo_SVD.fit(trainset)

pred_SVD = algo_SVD.test(unobserved_ratings)
rmse_score_svd = accuracy.rmse(pred_SVD)

print(f"For SVD model, the RMSE: {rmse_score_svd}")

RMSE: 0.3438
For SVD model, the RMSE: 0.3438193128761712


## Step 2
Now, we are interested not in whether the system properly predicts the rating
of these items, but rather whether the system gives the best recommendations
for each user.

To evaluate this, generate the top-k (with k = 10) recommendation
for each test user. 

In [110]:
#general method for top-k recommendations
from collections import defaultdict
from surprise.prediction_algorithms.predictions import Prediction
from typing import Dict, List
import numpy as np

def get_top_k(predictions: List[Prediction], 
              k: int) -> Dict[str, List]:
    """Compute the top-K recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        k(int): The number of recommendation to output for each user.
    Returns:
        A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """
    topk = defaultdict(list)
    
    for pred in predictions:
        uid = pred.uid 
        iid = pred.iid 
        est = pred.est 
        topk[uid].append((iid, est))
    
    # Sort the predictions
    for uid, user_ratings in topk.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        topk[uid] = user_ratings[:k]
    return topk

top10_knn = get_top_k(pred_KNN, k=10)
top10_svd = get_top_k(pred_SVD, k=10)

Based on the top-k recommendation list generated for
each user, and using the test data split
, compute:
• Hit rate, averaged across users.
• Precision@k, averaged across users
• Mean Average Precision (MAP@k)
• Mean Reciprocal Rank (MRR@k)
• Coverage

You need to convert 4 and 5 point ratings in the test split to binary labels, i.e., ratings
≤ 3 are mapped to 0 and ratings ≥ 4 are mapped to 1. For example, with Hit rate, if a user
gave a rating ≥ 4 to one of the top-k items we recommended (i.e., the item from the test split
is among our recommendations), then we consider that as a hit.


In [111]:
# using 3 as a threshold
test_data['new_label'] = test_data['rating'].apply(lambda x: 1 if x >= 3 else 0)
#test_data[test_data['new_label'] == 1]

In [112]:
import numpy as np
from __future__ import (absolute_import, division, print_function, unicode_literals)
from collections import defaultdict
from surprise import Dataset


def precision_at_k(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> Dict[str, float]:
    """Compute precision at k for each user
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        A dictionary where keys are user ids (str) and values are P@k (float) for each user.
    """
    
    precisions = defaultdict(float)
    
    # Only consider relevant items (rating ≥ 3.0)
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    
    for user, recommended_items in top_k.items():
        recommended_set = {item for item, _ in recommended_items[:k]}  # Take top-k items
        
        if user in relevant_items:
            num_relevant_at_k = len(recommended_set & relevant_items.get(user, set()))  # Intersection count
            if k > 0:  # Avoid division by zero
                precisions[user] = round(num_relevant_at_k / min(len(recommended_items), k), 3)  # Compute Precision@k

    return precisions



def mean_average_precision(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> float:
    """Compute mean average precision (MAP@k)
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        MAP@k (float)
    """
    
    average_precision_users = []
    
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()

    for user, recommended_items in top_k.items():
        relevant_set = relevant_items.get(user, set())  # Get relevant items, default to empty set
        
        num_relevant = 0
        precision_sum = 0.0
        
        for i, (item, _) in enumerate(recommended_items[:k]):  # Iterate over top-K items
            if item in relevant_set:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)  # Precision at each relevant item

        # Avoid division by zero
        avg_precision = precision_sum / min(len(recommended_items), k) if num_relevant > 0 else 0
        average_precision_users.append(avg_precision)

    return np.mean(average_precision_users) if average_precision_users else 0.0


def mean_reciprocal_rank(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> float:
    """Compute mean reciprocal rank (MRR@k)
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        MRR@k (float)
    """
    
    reciprocal_ranks = []
    
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()

    for user, recommended_items in top_k.items():
        relevant_set = relevant_items.get(user, set())  # Get relevant items, default to empty set
        found_relevant = False
        
        for i, (item, _) in enumerate(recommended_items[:k]):  # Iterate over top-K items
            if item in relevant_set:  # Find first relevant item
                reciprocal_ranks.append(1 / (i + 1))
                found_relevant = True
                break  # Stop after first relevant item

        if not found_relevant:
            reciprocal_ranks.append(0)  # Assign 0 if no relevant item is found

    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0



def hit_rate(top_k: Dict[str, List[str]],
             df_test: pd.DataFrame) -> float:
    """Compute the hit rate
    Args:
        top_k: A dictionary where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n (output of get_top_k())
        df_test: Pandas DataFrame containing user-item ratings in 
            the test split.
    Returns:
        The average hit rate
    """

    hits = 0
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    total_users = len(df_test[df_test['new_label'] == 1]['user_id'].unique())

    for user, recommended_items in top_k.items():
        recommended_set = {item for item, _ in recommended_items}  # Extract recommended item IDs
        if user in relevant_items:
            if recommended_set & relevant_items[user]:  # Check if there is any intersection
                hits += 1  

    return round(hits / total_users, 3) if total_users > 0 else 0.0



def coverage(top_k: Dict[str, List[str]], df_test: pd.DataFrame, df_train: pd.DataFrame) -> float:
    """
    Compute catalog coverage.

    Args:
        top_k: A dictionary where keys are user (raw) ids and values are lists of tuples:
               [(raw item id, rating estimation), ...] (output of get_top_k()).
        df_test: Pandas DataFrame containing the training data (user-item interactions).

    Returns:
        Coverage as a float (rounded to 3 decimals).
    """
    if not top_k:
        return 0.0  # No recommendations made

    recommended_items = {item for recommendations in top_k.values() for item, _ in recommendations}
    all_items = set(df_train["item_id"].unique()) | set(df_test["item_id"].unique()) 

    coverage_score = len(recommended_items) / len(all_items) if all_items else 0

    return round(coverage_score, 3)  # Round to 3 decimal places

### Evaluate KNN

In [113]:

print("Metrics for KNN CF:")
# PRECISION
precisions_nb = precision_at_k(top10_knn, test_data, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_knn, test_data, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_knn, test_data, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_knn, test_data)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_knn, test_data, train_data)
print("Coverage@10: {:.3f}".format(cover))

Metrics for KNN CF:
Averaged P@10: 0.012
MAP@10: 0.002
MRR@10: 0.019
Hit rate@10: 0.111
Coverage@10: 0.807


### Evaluate SVD

In [114]:
print("Metrics for SVD CF:")
# PRECISION
precisions_nb = precision_at_k(top10_svd, test_data, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_svd, test_data, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_svd, test_data, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_svd, test_data)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_svd, test_data, train_data)
print("Coverage@10: {:.3f}".format(cover))

Metrics for SVD CF:
Averaged P@10: 0.012
MAP@10: 0.002
MRR@10: 0.020
Hit rate@10: 0.115
Coverage@10: 0.255


In [115]:
#preserve result for KNN
prediction_KNN = {}
for prediction in pred_KNN:
    uid = prediction.uid
    iid = prediction.iid
    est = prediction.est
    if uid not in prediction_KNN:
        prediction_KNN[uid] = {}
    
    prediction_KNN[uid][iid] = est
import pickle

with open('prediction_KNN.pickle', 'wb') as file:
    pickle.dump(prediction_KNN, file)


### Evalute Baseline TopPop

In [116]:
top_items_list = list(zip(top_items['item_id'], top_items['avg_rating']))
top10_toppop = defaultdict(list)

for user in prediction_KNN:
    top10_toppop[user] = top_items_list

print("Metrics for TopPop:")
# PRECISION
precisions_nb = precision_at_k(top10_toppop, test_data, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_toppop, test_data, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_toppop, test_data, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_toppop, test_data)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_toppop, test_data, train_data)
print("Coverage@10: {:.3f}".format(cover))


Metrics for TopPop:
Averaged P@10: 0.005
MAP@10: 0.000
MRR@10: 0.005
Hit rate@10: 0.047
Coverage@10: 0.019


### long tail analysis

top20% and last20% user

In [117]:
from typing import Dict, List, Tuple
import pandas as pd

def hit_rate(top_k: Dict[str, List[tuple]], df_test: pd.DataFrame) -> float:
    hits = 0
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    total_users = len(relevant_items)

    for user, recommended_items in top_k.items():
        if user in relevant_items:
            recommended_set = {item for item, _ in recommended_items}
            if recommended_set & relevant_items[user]:
                hits += 1

    return round(hits / total_users, 3) if total_users > 0 else 0.0

def top_and_last_20_hit_rate(top_k: Dict[str, List[tuple]], df_test: pd.DataFrame, df_train: pd.DataFrame) -> Tuple[float, float]:
    user_interaction_counts = df_train.groupby('user_id').size().sort_values(ascending=False)
    num_users = len(user_interaction_counts)
    top_20_users = set(user_interaction_counts.head(int(0.2 * num_users)).index)
    last_20_users = set(user_interaction_counts.tail(int(0.2 * num_users)).index)

    df_test_top_20 = df_test[df_test['user_id'].isin(top_20_users)]
    df_test_last_20 = df_test[df_test['user_id'].isin(last_20_users)]

    top_20_hr = hit_rate(top_k, df_test_top_20)
    last_20_hr = hit_rate(top_k, df_test_last_20)

    return top_20_hr, last_20_hr

top20_hit_knn, last20_hit_knn = top_and_last_20_hit_rate(top10_knn, test_data, train_data)
top20_hit_svd, last20_hit_svd = top_and_last_20_hit_rate(top10_svd, test_data, train_data)
top20_hit_toppop, last20_hit_toppop = top_and_last_20_hit_rate(top10_toppop, test_data, train_data)

print("KNN Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_knn, last20_hit_knn))
print("SVD Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_svd, last20_hit_svd))
print("TopPop Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_toppop, last20_hit_toppop))



KNN Hit Rate - Top 20%: 0.050, Last 20%: 0.181
SVD Hit Rate - Top 20%: 0.100, Last 20%: 0.181
TopPop Hit Rate - Top 20%: 0.033, Last 20%: 0.069


top 20% and last 20% of items

In [118]:
from typing import Dict, List, Set, Tuple
def coverage(top_k: Dict[str, List[str]], relevant_items: Set[str]) -> float:

    recommended_items = {item for recs in top_k.values() for item, _ in recs}
    matched = recommended_items & relevant_items
    return round(len(matched)/len(relevant_items), 3) if relevant_items else 0

def get_item_groups(df_train: pd.DataFrame) -> Tuple[Set[str], Set[str]]:

    item_counts = df_train['item_id'].value_counts()
    split_idx = int(len(item_counts) * 0.2)
    return set(item_counts.head(split_idx).index), set(item_counts.tail(split_idx).index)

top_items, tail_items = get_item_groups(train_data)

# Calculate coverage for different groups
def calculate_group_coverage(top_k: Dict[str, List[str]], items: Set[str]) -> float:
    return coverage(top_k, items)



top20_cov_knn = calculate_group_coverage(top10_knn, top_items)
last20_cov_knn = calculate_group_coverage(top10_knn, tail_items)
print(f"KNN coverage - Top 20%: {top20_cov_knn}, Tail 20%: {last20_cov_knn}")

top20_cov_svd = calculate_group_coverage(top10_svd, top_items)
last20_cov_svd = calculate_group_coverage(top10_svd, tail_items)
print(f"SVD coverage - Top 20%: {top20_cov_svd}, Tail 20%: {last20_cov_svd}")

top20_cov_toppop = calculate_group_coverage(top10_toppop, top_items)
last20_cov_toppop = calculate_group_coverage(top10_toppop, tail_items)
print(f"TopPop coverage - Top 20%: {top20_cov_toppop}, Tail 20%: {last20_cov_toppop}")

KNN coverage - Top 20%: 0.99, Tail 20%: 0.564
SVD coverage - Top 20%: 0.277, Tail 20%: 0.188
TopPop coverage - Top 20%: 0.01, Tail 20%: 0.0


### Error analysis for the neighborhood-based CF:
•Select two users from the training set, one with high RR and one with
low RR. These are your reference users. Retrieve the 10 nearest neigh-
bours of each reference user. Print their rating history and analyse their
predictions. How much overlap is there between the rating history of the
reference users and their neighbours?
•For those users or items that your model performs poorly on (RR ≤0.05),
discuss the potential reasons behind.

In [119]:
from collections import defaultdict
from surprise import accuracy
import random

def calculate_rr(user_id, test_data, predictions, max_rank=100):
    relevant_items = set(test_data[test_data['user_id'] == user_id]['item_id'])
    user_predictions = sorted(
        [pred for pred in predictions if pred.uid == user_id],
        key=lambda x: x.est,
        reverse=True
    )[:max_rank]  
    for rank, pred in enumerate(user_predictions, 1):
        if pred.iid in relevant_items:
            return 1 / rank
    return 0

sampled_users = random.sample(list(set(pred.uid for pred in pred_KNN)), 800)
user_rr = {user: calculate_rr(user, test_data, pred_KNN) for user in sampled_users}

high_rr_user = max(user_rr, key=user_rr.get)
low_rr_user = min(user_rr, key=user_rr.get)
print(f"High RR User: {high_rr_user}")
print(f"Low RR User: {low_rr_user}")



High RR User: AF2MGAQBAT3E4XQC7NNAQFYG4MIQ
Low RR User: AEHWHU26Z2VEZQY3IEEW5HBIGX5Q


In [120]:
# 10 nearest neighbors
def get_neighbors(user_id, algo_knn, trainset):
    user_inner_id = trainset.to_inner_uid(user_id)
    neighbors = algo_knn.get_neighbors(user_inner_id, k=10)
    return [trainset.to_raw_uid(inner_id) for inner_id in neighbors]

print("Getting neighbors...")
high_rr_neighbors = get_neighbors(high_rr_user, algo_knn, trainset)
low_rr_neighbors = get_neighbors(low_rr_user, algo_knn, trainset)

# Print rating history
def print_rating_history(user_id, neighbors, trainset, max_items=10):
    print(f"Rating History for User {user_id} (top {max_items} items):")
    user_ratings = trainset.ur[trainset.to_inner_uid(user_id)][:max_items]
    for item_inner_id, rating in user_ratings:
        print(f"Item {trainset.to_raw_iid(item_inner_id)}: {rating}")
    print("Rating History for Neighbors (top 5 items each):")
    for neighbor in neighbors:
        neighbor_ratings = trainset.ur[trainset.to_inner_uid(neighbor)][:5]
        print(f"Neighbor {neighbor}:")
        for item_inner_id, rating in neighbor_ratings:
            print(f"Item {trainset.to_raw_iid(item_inner_id)}: {rating}")

print("Printing rating histories...")
print_rating_history(high_rr_user, high_rr_neighbors, trainset)
print_rating_history(low_rr_user, low_rr_neighbors, trainset)

# overlap in rating history
def analyze_overlap(user_id, neighbors, trainset):
    user_items = set(trainset.to_raw_iid(item_inner_id) for item_inner_id, _ in trainset.ur[trainset.to_inner_uid(user_id)])
    neighbor_items = set()
    for neighbor in neighbors:
        neighbor_items.update(trainset.to_raw_iid(item_inner_id) for item_inner_id, _ in trainset.ur[trainset.to_inner_uid(neighbor)])
    overlap = user_items.intersection(neighbor_items)
    print(f"Overlap rate in Rating History for User {user_id}: {len(overlap)/len(user_items)}")

analyze_overlap(high_rr_user, high_rr_neighbors, trainset)
analyze_overlap(low_rr_user, low_rr_neighbors, trainset)



Getting neighbors...
Printing rating histories...
Rating History for User AF2MGAQBAT3E4XQC7NNAQFYG4MIQ (top 10 items):
Item B000SHQ1QC: 4.0
Rating History for Neighbors (top 5 items each):
Neighbor AE23LDQTB7L76AP6E6WPBFVYL5DA:
Item B005M0MUQK: 4.0
Item B007T8CUNG: 5.0
Item B015IJIO5U: 5.0
Item B09NLV5LBK: 5.0
Item B0BSR996X8: 5.0
Neighbor AE23ZFVUOMPKR57BVSWXV34QLMVA:
Item B000NGVQKO: 5.0
Item B005PGGU9O: 3.0
Item B00CPLODUU: 5.0
Item B00RX5HQS4: 5.0
Item B00VSYN25M: 5.0
Neighbor AE2BV2H57ERXAPW7SOAXFLWA2S2Q:
Item B00EM5UOE6: 4.0
Item B00JL5I61A: 5.0
Item B015IJIO5U: 3.0
Item B06Y2W66F5: 1.0
Item B07T3GZVMD: 5.0
Neighbor AE2EQIUBKQCVSP55MNF2SQIASN2Q:
Item B003AYNHGW: 5.0
Item B00H35YIJE: 5.0
Item B00JL5I3AO: 5.0
Item B01DE4DTAG: 5.0
Item B08SJY4T7K: 5.0
Neighbor AE2NWSTL7JJJWOCBKZCZF6KDQIZQ:
Item B015WUVA5G: 5.0
Item B07226MCY6: 4.0
Item B0BRS6V8G4: 5.0
Neighbor AE2OQ55HLV5XO54DWLE4PB5XUNPA:
Item B0002E3FBA: 5.0
Item B079P9LDHN: 5.0
Item B07B16JL73: 4.0
Item B07D4QCBX4: 5.0
Item B07GF